In [ ]:
!pip install selenium
!pip install --upgrade selenium
!pip install webdriver_manager
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import os, time, random
from bs4 import BeautifulSoup
import urllib.request

SCROLL_PAUSE_TIME = 1

def chromeWebDriver():
    option = Options()
    option.add_argument('lang=ko_KR') # 언어 설정
    option.add_argument('disable-infobars')
    option.add_argument('--disable-extensions')
    option.add_experimental_option('detach', True) # 브라우저 안 닫히게
    option.add_experimental_option('excludeSwitches', ['enable-logging']) # 시스템 장치 에러 숨기기
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'
    
    option.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
    
    return driver

def collect_image(search_word, extract_img_count):
    url = 'https://www.google.co.kr'
    
    now = time.localtime()
    today_time = f'{now.tm_year}{now.tm_mon}{now.tm_mday}_{now.tm_hour}{now.tm_min}{now.tm_sec}'
    print(today_time)
    
    file_paht = 'C:\mask'
    
    os.chdir(file_paht)
    os.makedirs(file_paht+today_time+'_'+search_word)
    os.chdir(file_paht+today_time+'_'+search_word)
    file_save_dir = file_paht+today_time+'_'+search_word
    print(file_save_dir)
    
    driver = webdriver.Chrome('E:\Project\chromedriver.exe')
    driver.get(url)
    time.sleep(random.uniform(2,3))
    elem_q = driver.find_element(By.NAME, 'q')
    elem_q.send_keys(search_word)
    elem_q.submit()
    
    driver.find_element(By.LINK_TEXT, '이미지').click()
    time.sleep(SCROLL_PAUSE_TIME)
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # 페이지 스크롤 다운
    while True:
        # 스크롤을 내리는 자바스크립트 코드를 실행시킴
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # 페이지가 로드될 때까지 기다림
        time.sleep(SCROLL_PAUSE_TIME)
        # 길어진 스크롤 높이 값을 new_height값으로 지정
        new_height = driver.execute_script("return document.body.scrollHeight")
        # 더 이상 내릴게 없다.
        if new_height == last_height:
            try:
                driver.find_element_by_css_selector(".mye4qd").click()
            # 버튼이 없을 시 에러가 발생하면 반복문을 빠져나가도록 함.
            except:
                break
        last_height = new_height
    
    
    file_no = 1
    count = 1
    img_src = []
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    imgs = driver.find_elements(By.CSS_SELECTOR, '#islrg > div.islrc > div a.wXeWr.islib.nfEiy')
    print(len(imgs))
    
    for img in imgs:
        img_src1 = img.click()
        try:
            img_src2 = driver.find_element(By.CSS_SELECTOR, '#Sva75c > div > div > div.pxAole > div.tvh9oe.BIB1wf > c-wiz > div > div.OUZ5W > div.zjoqD > div.qdnLaf.isv-id > div > a')
        except Exception:
            continue
        time.sleep(random.uniform(0.2, 0.5))
        img_src3 = img_src2.find_element(By.TAG_NAME, 'img').get_attribute('src')
        if img_src3[:4] != 'http':
            continue
        print(count, img_src3, '\n')
        
        img_src.append(img_src3)
        if count == extract_img_count + 10: # 이미지 에러 대비해서 입력 숫자보다 크게 잡음
            break            
        count += 1
        
    print(f'\n{"="*10} 추출한 전체 리스트 {"="*10}\n{img_src}\n\n{"="*10}총 {len(img_src)}개 추출함{"="*10}\n')
        
    for i in range(len(img_src)):
        extension = img_src[i].split('.')[-1]
        ext = ''
        print(extension)
        if extension in ('jpg', 'JPG',' jpeg', 'JPEG', 'png', 'PNG', 'gif', 'GIF'):
            ext = '.'+extension
        else:
            ext = '.jpg'
        
        try:
            urllib.request.urlretrieve(img_src[i], str(file_no).zfill(3) + ext)
            print(img_src[i])
        except Exception:
            continue
        
        print(f'{file_no}번째 이미지 저장-----')
        file_no += 1
        
        if file_no - 1 == extract_img_count:
            break
        
    driver.close()
            
    
if __name__ == '__main__':
    collect_image('연예인 셀카', 500)